In [101]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as dt
import pandas as pd 
import numpy as np
import torch.utils.data as data_utils

# Input and Hyperparameters #

In [152]:
data_path = "./US_length.csv"
learning_rate = 0.001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
split = 0.8
batch_size = 50
num_epochs = 50
subset_val = 1000

# Data Cleaning #

In [153]:
df_main = pd.read_csv(data_path, sep = ',')
#subset data for testing ,use the first 10000
df = df_main[:subset_val]
secs = df['sec'].values
mins = df['min'].values
hours = df['hour'].values
total_time = [s+60*m+3600*h for s,m,h in zip(secs, mins, hours)]
df['total_time'] = total_time
df_features=df[['title', 'total_time']]

C:\Users\ChuckDaddy\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [154]:
df_features['title_upper_count'] = df_features['title'].str.findall(r'[A-Z]').str.len()
df_features['tag_count'] = [len(s.split('|')) for s in df['tags'].values]
df_features['category_id'] = df_main['category_id']
df_label = pd.DataFrame()
df_label['views'] = df['views']
df_features = df_features.drop(columns = 'title')

C:\Users\ChuckDaddy\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\ChuckDaddy\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\ChuckDaddy\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [155]:
df_features.head()


,total_time,title_upper_count,tag_count,category_id
0,678,28,1,22
1,1431,11,4,24
2,569,8,23,23
3,430,4,27,24
4,733,12,14,24


In [156]:
df_label.head()


,views
0,748374
1,2418783
2,3191434
3,343168
4,2095731


# Setup Dataloader #

In [157]:
tv_split = int(len(df_features)*split)
train_set = df_features[:tv_split]
train_label = df_label[:tv_split]
validation_set = df_features[tv_split:]
validation_label = df_label[tv_split:]


In [158]:
class Dataset(dt.Dataset):
    def __init__(self, feature, labels):
        self.labels = labels
        self.feature = feature
    def __len__(self):
        return(len(self.feature))
    def __getitem__(self, idx):
        cur_feature = self.feature.iloc[idx]
        cur_feature = np.array(cur_feature)
        cur_label = self.labels.iloc[idx]
        cur_label = np.array(cur_label)
        sample = (cur_feature, cur_label)
        return sample

In [159]:
train = Dataset(train_set, train_label)
train_loader = data_utils.DataLoader(train, batch_size, shuffle = True)
test = Dataset(validation_set, validation_label)
test_loader = data_utils.DataLoader(test, batch_size, shuffle = True)

# Define Neural Network

In [160]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size).double() 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes).double()
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        return out

# Main #

In [161]:
input_size = len(df_features.iloc[0])
output_size = len(df_label.columns)
hidden_size = int(np.ceil((input_size+output_size)/2))
model = NeuralNet(input_size, hidden_size, output_size).to(device)
# Loss and optimizer
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  
# Train the model
total_step = len(train_loader)
print(total_step)
loss = torch.tensor(0.0)
for epoch in range(num_epochs):
    curr_epoch_loss = 0.0
    for i, (feature, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        # Move tensors to the configured device
        feature = torch.tensor(feature,  dtype = torch.float64).to(device)
        label = torch.tensor(labels,  dtype = torch.float64).to(device)

#         # Forward pass/
        outputs = model(feature)
#         print(outputs, label)
        loss = criterion(outputs, label)
        curr_epoch_loss+=loss.data.item()
        loss.backward()
        optimizer.step()
#         print(loss.data.item())
    print(curr_epoch_loss)
#         
        
#         # Backward and optimize
#         
#         loss.backward()
#         optimizer.step()
        
        #if (i+1) % 100 == 0:
#         print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.1f}' 
#                    .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model

16


C:\Users\ChuckDaddy\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\ChuckDaddy\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


16374260.457335453
16374050.840803565
16373845.024876758
16373629.464946207
16373377.94297805
16373145.237324541
16372880.877013382
16372572.205949882
16372303.949740438
16371974.380272534
16371639.742622696
16371277.122486835
16370917.688334871
16370504.963623151
16370099.99386009
16369681.44328047
16369222.231477987
16368766.626964625
16368269.319106346
16367742.159902023
16367240.558703465
16366644.017858185
16366093.249688055
16365490.718819125
16364895.782396294
16364234.984079622
16363607.079447798
16362925.61543157
16362494.853755794
16362240.741850175
16361954.461927796
16361722.19298501
16361447.729358127
16361175.031249436
16360874.554855995
16360624.115839405
16360305.348061278
16360002.343249673
16359713.437839394
16359427.00361175
16359068.12766912
16358744.086878307
16358432.152679514
16358043.439361759
16357732.077850986
16357340.678663518
16357025.845812142
16356596.989307
16356230.69988333
16355837.920127504


In [162]:
# for i, (feature, labels) in enumerate(train_loader):  
#     if(i >10):
#         break
#     print((type(feature)))
#     print(labels)